<a href="https://colab.research.google.com/github/mdjamina/m1_ml_lang_detector/blob/main/data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus
[Tatoeba](https://tatoeba.org/fr/downloads
) est une collection de phrases et de traductions.

## Téléchargement du corpus

In [1]:
!wget https://iso639-3.sil.org/sites/iso639-3/files/downloads/iso-639-3-macrolanguages.tab

--2022-03-12 22:13:36--  https://iso639-3.sil.org/sites/iso639-3/files/downloads/iso-639-3-macrolanguages.tab
Resolving iso639-3.sil.org (iso639-3.sil.org)... 104.22.10.254, 172.67.29.248, 104.22.11.254, ...
Connecting to iso639-3.sil.org (iso639-3.sil.org)|104.22.10.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5012 (4.9K)
Saving to: ‘iso-639-3-macrolanguages.tab’

iso-639-3-macrolang 100%[===================>]   4.89K  --.-KB/s    in 0s      

2022-03-12 22:13:36 (53.4 MB/s) - ‘iso-639-3-macrolanguages.tab’ saved [5012/5012]



In [2]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2

--2022-03-12 22:13:39--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160225890 (153M) [application/octet-stream]
Saving to: ‘sentences.tar.bz2’

sentences.tar.bz2   100%[===================>] 152.80M  32.1MB/s    in 5.5s    

2022-03-12 22:13:45 (27.9 MB/s) - ‘sentences.tar.bz2’ saved [160225890/160225890]



Extraction de l'archive téléchargée 

## Chargement du corpus

In [3]:
import pandas as pd

data = pd.read_csv('sentences.tar.bz2', sep='\t', header=None,compression='bz2')
data.columns = ['id','I_Id', 'content']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [51]:
data.shape

(10262659, 3)

## Nettoyage des données

In [52]:
#suppression des valeurs NAN
data.dropna(subset = ["I_Id"], inplace=True)
#identification des colonnes qui contiennent des valeurs NAN (non renseignées)
data.isnull().any()

id         False
I_Id       False
content    False
dtype: bool

## regrouppement des dialects par langue principale

In [113]:
langs ={'eng':'English','pol':'Polish','deu':'German','fra':'French','spa':'Spanish','ita':'Italian'
,'tur':'Turkish','por':'Portuguese','rus':'Russian','ukr':'Ukrainian','nld':'Dutch','bul':'Bulgarian'
,'ell':'Greek','swe':'Swedish','hun':'Hungarian','gle':'Irish','lav':'Latvian','dan':'Danish','fin':'Finnish',
'ara':'Arabic','heb':'Hebrew','zho':'Chinese','hin':'Hindi','jpn':'Japanese','fas':'Persian','kor':'Korean',
'hye':'Armenian','swa':'Swahili','ber':'Berber','kab':'Kabyle','ces':'Czech','lat':'Latin',
'nor':'Norwegian','ron':'Moldavian, Moldovan, Romanian','slk':'Slovak','hbs':'Serbo-Croatian','mkd':'Macedonian',
'vie':'Vietnamese','est':'Estonian','tha':'Thai'}


len(langs)


40

In [ ]:
#TODO
# site iso langues avec leur regroupement
# telecharger fichier csv(iso) contient : langue : arb=dz, ara=egy ..,  le telecharger avec  pandas data frames 

In [ ]:
import pandas as pd
macro_lang = pd.read_csv('https://raw.githubusercontent.com/mdjamina/m1_ml_lang_detector/main/data/iso-639-3-macrolanguages.tab', sep='\t')
macro_lang.columns = ['lang',	'I_Id',	'I_Status']

macro_lang

In [89]:
#jointure entre la table des sentences et la table macrolanguages
df = pd.merge(data,macro_lang, how='left',on='I_Id')
df.lang.fillna(df.I_Id, inplace=True)

#suppression des columns 'I_Status', 'id' et 'I_Id'
df = df.drop(columns=['I_Status', 'id','I_Id'])

#df[~df['lang'].isin(macro_lang['lang'])]

#garder que les langues selectionnées dans la liste langs
#df=df[df['lang'].isin(langs.keys())]

df.shape


(10262659, 2)

In [91]:
df_counts = df.groupby(['lang']).count()

In [74]:


#df_counts[(df_counts['content']>50000) ]
 
to_reduce0 = df[df['lang'].isin(df_counts[(df_counts['content']>=220000) & (df_counts['content']<500000) ].index)]
to_reduce1 = df[df['lang'].isin(df_counts[(df_counts['content']>=500000) & (df_counts['content']<1000000) ].index)]
to_reduce2 = df[df['lang'].isin(df_counts[(df_counts['content']>=1000000 ) ].index)]

df = df[df['lang'].isin(df_counts[(df_counts['content']<220000) ].index)]


In [ ]:
df_counts[(df_counts['content']<=1000) & (df_counts['content']>250) & (~df_counts.index.isin(langs.keys()))]

In [76]:
to_reduce0.groupby(['lang']).count()

,content
lang,
fra,497104
hun,353702
por,394187
spa,367512


In [77]:
to_reduce1.groupby(['lang']).count()

,content
lang,
ber,589554
deu,581093
ita,802842
kab,513886
rus,903431
tur,717173


In [78]:
to_reduce2.groupby(['lang']).count()

,content
lang,
eng,1572075


In [79]:
from sklearn.model_selection import train_test_split 
#on prends que "




In [84]:
content_reduced0, _, lang_reduced0, _ = train_test_split(to_reduce0['content'], to_reduce0['lang'], test_size=0.45, random_state=42) 
reduced0 = pd.DataFrame({'content':content_reduced0,'lang':lang_reduced0})


reduced0.groupby(['lang']).count()

,content
lang,
fra,273358
hun,194765
por,216536
spa,202218


In [85]:
content_reduced1, _, lang_reduced1, _ = train_test_split(to_reduce1['content'], to_reduce1['lang'], test_size=0.66, random_state=42) 

reduced1 = pd.DataFrame({'content':content_reduced1,'lang':lang_reduced1})


reduced1.groupby(['lang']).count()

,content
lang,
ber,200615
deu,197317
ita,273066
kab,174590
rus,307411
tur,243713


In [86]:
content_reduced2, _, lang_reduced2, _ = train_test_split(to_reduce2['content'], to_reduce2['lang'], test_size=0.80, random_state=42) 
reduced2 = pd.DataFrame({'content':content_reduced2,'lang':lang_reduced2})


reduced2.groupby(['lang']).count()

,content
lang,
eng,314415


In [87]:

df = df.append(reduced0)
df = df.append(reduced1)
df = df.append(reduced2)
df.groupby(['lang']).count()

,content
lang,
ara,42128
ber,200615
bul,24521
dan,53561
deu,197317
ell,34791
eng,314415
fas,24783
fin,138867


In [88]:
df.to_pickle('data.pkl')